In [1]:
import csv
import os
import pandas as pd
from configparser import ConfigParser
from mysql.connector import MySQLConnection, Error

# Utilities

In [2]:
def read_config(config_file = 'config.ini', section = 'mysql'):
    """
    Read the configuration file config_file with the given section.
    If successful, return the configuration as a dictionary,
    else raise an exception.
    """
    parser = ConfigParser()
    
    # Does the configuration file exist?
    if os.path.isfile(config_file):
        parser.read(config_file)
    else:
        raise Exception(f"Configuration file '{config_file}' "
                        "doesn't exist.")
    
    config = {}
    
    if parser.has_section(section):
        # Parse the configuration file.
        items = parser.items(section)
        
        # Construct the parameter dictionary.
        for item in items:
            config[item[0]] = item[1]
            
    else:
        raise Exception(f'Section [{section}] missing ' + \
                        f'in config file {config_file}')
    
    return config

In [3]:
def make_connection(config_file = 'config.ini', section = 'mysql'):
    """
    Make a database connection with the configuration file config_file
    with the given section. If successful, return the connection,
    else raise an exception.
    """
    try:
        db_config = read_config(config_file, section)
        conn = MySQLConnection(**db_config)

        if conn.is_connected():
            return conn

    except Error as e:
        raise Exception(f'Connection failed: {e}')

In [4]:
def dataframe_query(conn, sql):
    """
    Use the database connection conn to execute
    the SQL code. Return the resulting row count
    and the rows as a dataframe or (0, None) 
    if there were no rows. If the query failed,
    raise an exception.
    """
    try:
        cursor = conn.cursor()
        cursor.execute(sql)

        rows  = cursor.fetchall()
        count = cursor.rowcount

        if count > 0:

            # Get the names of the columns.
            columns = cursor.description
            column_names = [column_info[0] 
                            for column_info in columns]

            # Return the query results in a dataframe.
            df = DataFrame(rows)
            df.columns = column_names
            cursor.close()
            return count, df

        else:
            cursor.close()
            return 0, None
        
    except Error as e:
        raise Exception(f'Query failed: {e}')

In [5]:
conn = make_connection(config_file='sales.ini')
cursor = conn.cursor()


# Tables

In [6]:
cursor.execute('DROP TABLE IF EXISTS customers')

sql = ( """
        CREATE TABLE customers
        (
            cCode    varchar(255),
            cName    varchar(255),
            cType    varchar(255),
            PRIMARY KEY(cCode)
        )
        """
      )

cursor.execute(sql)
# Primary Key: cCode (customer_code)

In [7]:
cursor.execute('DROP TABLE IF EXISTS markets')

sql = ( """
        CREATE TABLE markets
        (
            mCode    varchar(255),
            mName   varchar(255),
            zone    varchar(255),
            PRIMARY KEY(mCode)
        )
        """
      )

cursor.execute(sql)
# Primary Key: mCode (market_code)

In [8]:
cursor.execute('DROP TABLE IF EXISTS products')

sql = ( """
        CREATE TABLE products
        (
            pCode    varchar(255),
            pType    varchar(255),
            PRIMARY KEY(pCode)
        )
        """
      )

cursor.execute(sql)
# Primary Key: pCode (product_code)

In [9]:
cursor.execute('DROP TABLE IF EXISTS transaction')

sql = ( """
        CREATE TABLE transaction
        (
            ID_column int AUTO_INCREMENT,
            pCode    varchar(255),
            cCode   varchar(255),
            mCode    varchar(255),
            date    date, 
            salesQ   double,
            salesA double,
            currency char(255),
            PRIMARY KEY(ID_column)
        )
        """
      )

cursor.execute(sql)
# Primary Key: ID_column
# Foreign Keys: pCode, cCode, mCode

# Inserts

In [10]:
sql_trans = ("""
                INSERT INTO transaction(pCode, cCode,mCode,date,salesQ,salesA,currency)
                VALUES (%s,%s,%s,%s,%s,%s,%s)
                """
               )
sql_cust = ("""
                INSERT INTO customers
                VALUES (%s,%s,%s)
                """
               )
sql_prod = ("""
                INSERT INTO products
                VALUES (%s,%s)
                """
               )
sql_mark = ("""
                INSERT INTO markets
                VALUES (%s,%s,%s)
                """
               )

In [11]:
# Inserting into transaction
count = 0
first = True
with open('transactions.csv', newline='') as csv_file:
    data = csv.reader(csv_file,delimiter = ',', quotechar='"')
    for row in data:
        if not first:
            count+=1
            if count == 99:
                break
            cursor.execute(sql_trans, row)
        first = False
conn.commit()

In [12]:
# Inserting into customers
count = 0
first = True
with open('customers.csv', newline='') as csv_file:
    data = csv.reader(csv_file,delimiter = ',', quotechar='"')
    for row in data:
        if not first:
            count+=1
            if count == 99:
                break
            cursor.execute(sql_cust, row)
        first = False
conn.commit()

In [13]:
# Inserting into products
count = 0
first = True
with open('products.csv', newline='') as csv_file:
    data = csv.reader(csv_file,delimiter = ',', quotechar='"')
    for row in data:
        if not first:
            count+=1
            if count == 99:
                break
            cursor.execute(sql_prod, row)
        first = False
conn.commit()

In [14]:
# Inserting into markets
count = 0
first = True
with open('markets.csv', newline='') as csv_file:
    data = csv.reader(csv_file,delimiter = ',', quotechar='"')
    for row in data:
        if not first:
            count+=1
            if count == 99:
                break
            if row[2] != '':
                cursor.execute(sql_mark, row)
        first = False
conn.commit()

# DataFrames

In [15]:
# DataFrame for customers
select_c = ("""
            SELECT * FROM customers LIMIT 25
            """)
cursor.execute(select_c)
df1 = pd.DataFrame(cursor.fetchall(), columns = ["customer_code", "customer_name","customer_type"])
display(df1)
conn.commit()

customer_code             customer_name   customer_type
0         Cus001              Surge Stores  Brick & Mortar
1         Cus002              Nomad Stores  Brick & Mortar
2         Cus003              Excel Stores  Brick & Mortar
3         Cus004            Surface Stores  Brick & Mortar
4         Cus005            Premium Stores  Brick & Mortar
5         Cus006     Electricalsara Stores  Brick & Mortar
6         Cus007               Info Stores  Brick & Mortar
7         Cus008          Acclaimed Stores  Brick & Mortar
8         Cus009   Electricalsquipo Stores  Brick & Mortar
9         Cus010              Atlas Stores  Brick & Mortar
10        Cus011           Flawless Stores  Brick & Mortar
11        Cus012        Integration Stores  Brick & Mortar
12        Cus013              Unity Stores  Brick & Mortar
13        Cus014            Forward Stores  Brick & Mortar
14        Cus015     Electricalsbea Stores  Brick & Mortar
15        Cus016              Logic Stores  Brick & Mortar
16        Cus017               Epic Stores  Brick & Mortar
17        Cus018   Electricalslance Stores  Brick & Mortar
18        Cus019  Electricalsopedia Stores  Brick & Mortar
19        Cus020                     Nixon      E-Commerce
20        Cus021                   Modular      E-Commerce
21        Cus022        Electricalslytical      E-Commerce
22        Cus023                     Sound      E-Commerce
23        Cus024                     Power      E-Commerce
24        Cus025                      Path      E-Commerce

In [16]:
# DataFrame for markets
select_m = ("""
            SELECT * FROM markets LIMIT 25
            """)
cursor.execute(select_m)
df2 = pd.DataFrame(cursor.fetchall(), columns = ["market_code", "market_name", "zone"])
display(df2)
conn.commit()

market_code   market_name     zone
0      Mark001       Chennai    South
1      Mark002        Mumbai  Central
2      Mark003     Ahmedabad    North
3      Mark004     Delhi NCR    North
4      Mark005        Kanpur    North
5      Mark006     Bengaluru    South
6      Mark007        Bhopal  Central
7      Mark008       Lucknow    North
8      Mark009         Patna    North
9      Mark010         Kochi    South
10     Mark011        Nagpur  Central
11     Mark012         Surat    North
12     Mark013        Bhopal  Central
13     Mark014     Hyderabad    South
14     Mark015  Bhubaneshwar    South

In [17]:
# DataFrame for products
select_p = ("""
            SELECT * FROM products LIMIT 25
            """)
cursor.execute(select_p)
df3 = pd.DataFrame(cursor.fetchall(), columns = ["product_code", "produce_type"])
display(df3)
conn.commit()

product_code    produce_type
0       Prod001     Own Brand\r
1       Prod002     Own Brand\r
2       Prod003     Own Brand\r
3       Prod004     Own Brand\r
4       Prod005     Own Brand\r
5       Prod006     Own Brand\r
6       Prod007     Own Brand\r
7       Prod008     Own Brand\r
8       Prod009     Own Brand\r
9       Prod010     Own Brand\r
10      Prod011     Own Brand\r
11      Prod012     Own Brand\r
12      Prod013     Own Brand\r
13      Prod014     Own Brand\r
14      Prod015     Own Brand\r
15      Prod016     Own Brand\r
16      Prod017     Own Brand\r
17      Prod018     Own Brand\r
18      Prod019     Own Brand\r
19      Prod020     Own Brand\r
20      Prod021     Own Brand\r
21      Prod022  Distribution\r
22      Prod023  Distribution\r
23      Prod024  Distribution\r
24      Prod025  Distribution\r

In [18]:
# DataFrame for transaction
select_t = ("""
            SELECT * FROM transaction LIMIT 25
            """)
cursor.execute(select_t)
df4 = pd.DataFrame(cursor.fetchall(), columns = ["Index","product_code", "customer_code", "market_code", 
                                                 "order_date", "sales_quantity", "sales_amount", "currency"])
display(df4)
conn.commit()

Index product_code customer_code market_code  order_date  sales_quantity  \
0       1      Prod001        Cus001     Mark001  2017-10-10           100.0   
1       2      Prod001        Cus002     Mark002  2018-05-08             3.0   
2       3      Prod002        Cus003     Mark003  2018-04-06             1.0   
3       4      Prod002        Cus003     Mark003  2018-04-11             1.0   
4       5      Prod002        Cus004     Mark003  2018-06-18             6.0   
5       6      Prod003        Cus005     Mark004  2017-11-20            59.0   
6       7      Prod003        Cus005     Mark004  2017-11-22            36.0   
7       8      Prod003        Cus005     Mark004  2017-11-23            39.0   
8       9      Prod003        Cus005     Mark004  2017-11-27            35.0   
9      10      Prod003        Cus005     Mark004  2017-11-28           310.0   
10     11      Prod003        Cus005     Mark004  2017-11-29           184.0   
11     12      Prod003        Cus005     Mark004  2017-11-30            35.0   
12     13      Prod004        Cus005     Mark004  2017-11-29            17.0   
13     14      Prod004        Cus005     Mark004  2017-12-19             1.0   
14     15      Prod005        Cus005     Mark004  2018-08-07             5.0   
15     16      Prod003        Cus006     Mark004  2017-12-04            58.0   
16     17      Prod005        Cus006     Mark004  2018-06-29            38.0   
17     18      Prod005        Cus006     Mark004  2018-07-02            93.0   
18     19      Prod005        Cus006     Mark004  2018-07-03            79.0   
19     20      Prod005        Cus006     Mark004  2018-07-04             1.0   
20     21      Prod005        Cus006     Mark004  2018-07-06             3.0   
21     22      Prod005        Cus006     Mark004  2018-07-13             1.0   
22     23      Prod005        Cus006     Mark004  2019-06-07            20.0   
23     24      Prod005        Cus006     Mark004  2019-07-29            81.0   
24     25      Prod005        Cus006     Mark004  2019-08-01             5.0   

    sales_amount currency  
0        41241.0      INR  
1           -1.0      INR  
2          875.0      INR  
3          583.0      INR  
4         7176.0      INR  
5          500.0      USD  
6          250.0      USD  
7        21412.0      INR  
8        19213.0      INR  
9       170185.0      INR  
10      101194.0      INR  
11       19213.0      INR  
12        9426.0      INR  
13         218.0      INR  
14        3093.0      INR  
15       30306.0      INR  
16       52319.0      INR  
17      126296.0      INR  
18      107500.0      INR  
19         273.0      INR  
20        3574.0      INR  
21         273.0      INR  
22       23403.0      INR  
23       76329.0      INR  
24        4542.0      INR

# One to One


In [19]:
oto = ("""
        SELECT t.pCode, c.cCode, c.cName, t.date 
        FROM transaction as t, customers as c
        WHERE t.salesA = 41241
        and t.cCode = c.cCode""")
cursor.execute(oto)
results = cursor.fetchall()
for row in results:
    print(row)
conn.commit()

('Prod001', 'Cus001', 'Surge Stores', datetime.date(2017, 10, 10))


# One to Many

In [20]:
otm = ("""
        SELECT t.pCode,t.cCode, t.date
        FROM transaction as t, products as p
        WHERE p.pCode = 'Prod001'
        AND p.pCode = t.pCode""")
cursor.execute(otm)
results = cursor.fetchall()
for row in results:
    print(row)
conn.commit()

('Prod001', 'Cus001', datetime.date(2017, 10, 10))
('Prod001', 'Cus002', datetime.date(2018, 5, 8))


In [21]:
otmt = ("""
        SELECT t.cCode, t.pCode
        FROM transaction t, customers c
        WHERE t.cCode = c.cCode
        AND c.cName = 'Premium Stores'
        """)
cursor.execute(otmt)
results = cursor.fetchall()
for row in results:
    print(row)
conn.commit()

('Cus005', 'Prod003')
('Cus005', 'Prod003')
('Cus005', 'Prod003')
('Cus005', 'Prod003')
('Cus005', 'Prod003')
('Cus005', 'Prod003')
('Cus005', 'Prod003')
('Cus005', 'Prod004')
('Cus005', 'Prod004')
('Cus005', 'Prod005')


In [22]:
otot = ("""SELECT c.cCode, t.salesA
        FROM customers as c, transaction as t, markets as m
        WHERE m.mCode = 'Mark001' 
        AND c.cCode=t.cCode
        AND c.cCode = 'Cus005'""")
cursor.execute(otot)
results = cursor.fetchall()
for row in results:
    print(row)
conn.commit()

('Cus005', 500.0)
('Cus005', 250.0)
('Cus005', 21412.0)
('Cus005', 19213.0)
('Cus005', 170185.0)
('Cus005', 101194.0)
('Cus005', 19213.0)
('Cus005', 9426.0)
('Cus005', 218.0)
('Cus005', 3093.0)


# Many to Many

In [23]:
mtm = ("""
        SELECT t.pCode, c.cName, m.mCode, m.zone, m.mName 
        FROM customers c, markets m, transaction t
        WHERE c.cName IN ("Excel Stores", "Surge Stores", "Surface Stores")
        AND c.cCode = t.cCode
        AND m.mCode = t.mCode
        
        """)

cursor.execute(mtm)
results = cursor.fetchall()
for row in results:
    print(row)
conn.commit()

('Prod001', 'Surge Stores', 'Mark001', 'South', 'Chennai')
('Prod002', 'Excel Stores', 'Mark003', 'North', 'Ahmedabad')
('Prod002', 'Excel Stores', 'Mark003', 'North', 'Ahmedabad')
('Prod002', 'Surface Stores', 'Mark003', 'North', 'Ahmedabad')


In [24]:
cursor.close()
conn.close()